In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# define vars
PATH_DEVELOP = 'test_dev'
PATH_PRODUCTION = 'test_prod'
START_DAY = 1
START_MON = 1
START_YEAR = 2022

END_DAY = 1
END_MON = 7
END_YEAR = 2023

# Define what channel id you will use
channel_id = 'UCWdGgeyOnCOGnvhNdbVl1dQ'

# Define the API key from Google Cloud Console
API_KEY = ""



In [3]:
PATH_DEV = "/content/drive/My Drive/" + PATH_DEVELOP + '/'
PATH_PROD = "/content/drive/My Drive/" + PATH_PRODUCTION + '/'

!mkdir -p "{PATH_DEV}"
!mkdir -p "{PATH_PROD}"

In [4]:
import re
import os
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import time

In [ ]:
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=API_KEY)

def get_all_videos_from_channel(channel_id, start_date, end_date):
    video_ids = []
    video_titles = []
    video_dates = []
    token = None
    formatted_date = f"{END_YEAR}-{END_MON:02d}-{END_DAY:02d}T00:00:00Z"

    stop_search = 1

    while stop_search:

      search_params = {
          'channelId': channel_id,
          'type': "video",
          'part': "id,snippet",
          'maxResults': 50,
          'order': 'date',
          'publishedBefore' : formatted_date,
          'pageToken': token
      }



      search_response = youtube.search().list(**search_params).execute()

      for search_result in search_response.get("items", []):
        publish_date = search_result["snippet"]["publishedAt"]

        video_date = datetime.strptime(publish_date, '%Y-%m-%dT%H:%M:%SZ')
        if(start_date <= video_date <= end_date):
          video_ids.append(search_result["id"]["videoId"])
          video_titles.append(search_result["snippet"]["title"])
          video_dates.append(publish_date)
        if(start_date > video_date):
          stop_search = 0

        print(video_date)
      try:
        print(search_response["nextPageToken"])
        token = search_response["nextPageToken"]
      except:
        print('Não existe mais token ultima pagina obtida!')
        stop_search = 0


    df = pd.DataFrame({
        'Video ID': video_ids,
        'Video Title': video_titles,
        'Post Date': video_dates
    })

    return df

start_date = datetime(START_YEAR, START_MON, START_DAY)
end_date = datetime(END_YEAR, END_MON, END_DAY)

df = get_all_videos_from_channel(channel_id, start_date, end_date)
df.to_csv(PATH_DEV + 'index.csv',index=False)

In [6]:
def get_all_comments_from_video(video_id):
    comments = []
    token = None

    while True:
        search_params = {
            'videoId': video_id,
            'part': "snippet",
            'maxResults': 50,
            'pageToken': token
        }

        search_response = youtube.commentThreads().list(**search_params).execute()

        for item in search_response.get('items', []):
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append([
                comment['authorDisplayName'],
                comment['publishedAt'],
                comment['updatedAt'],
                comment['likeCount'],
                comment['textDisplay']
            ])

        try:
          token = search_response["nextPageToken"]
        except:
          break

    df = pd.DataFrame(comments, columns=['author', 'published_at', 'updated_at', 'like_count', 'text'])
    return df


In [ ]:
df_index = pd.read_csv(PATH_DEV + 'index.csv')

lista_csv = !ls "{PATH_DEV}"
processed_list = []
for item in lista_csv:
    split_items = item.split('\t')
    processed_list.extend([s.replace('.csv', '') for s in split_items])

file_list = []

for item in processed_list:
    file_list.extend(item.split())

lista_id = df_index['Video ID']

missing_ids = [vid_id for vid_id in lista_id if vid_id not in file_list]
len(missing_ids)
for id_nao_processado in missing_ids:
  print(id_nao_processado)
  df = get_all_comments_from_video(id_nao_processado)
  df.to_csv(PATH_DEV + id_nao_processado + '.csv',index=False)

In [ ]:
df_index = pd.read_csv(PATH_DEV + 'index.csv')

for index, row in df_index.iterrows():

    try:
      read_file = pd.read_csv(PATH_DEV + row['Video ID'] + '.csv', engine='python')
      date_obj = datetime.strptime(row['Post Date'], "%Y-%m-%dT%H:%M:%SZ")
      formatted_date = date_obj.strftime("%Y%m%d")
      safe_title = re.sub(r'[^a-zA-Z0-9 ]', '', row['Video Title'])
      file_name = f'{safe_title.replace(" ", "_")}'
      new_file_name = formatted_date + ';' + file_name + ';' + row['Video ID'] + ';' + f'{str(len(read_file) -1)}'+'.csv'
      read_file.to_csv(PATH_PROD + new_file_name,index=False)
      print(PATH_PROD + new_file_name + ' --> Arquivo inserido com sucesso')
    except:
      print(PATH_DEV + row['Video ID'] + '.csv' + ' --> Arquivo não foi encontrado')